In [1]:
from pylab import plot,show
import numpy as np
import pandas as pd
from numpy import linalg as LA
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import spsolve_triangular
plt.style.use('seaborn-white')

# Acoustic Wave Equation in Time Domain

The elastic wave equation can be stated as follows,
\begin{equation}
\frac{\partial^2 u}{\partial t^2}=c^2 \nabla^2 u+f,\qquad x\in \Omega \subset \mathbb{R}^d,~t\in(T_1,T_2]
\qquad\qquad\qquad\qquad (1)
\end{equation}
where $\Delta$ is *the Laplacian*, $f$ is a *forcing function* (for example our source) and $c$ is *the wave velocity* at which the time and spatially varying wave $u$ propagates.

* [Finite-Difference Methods for the One Dimensional Wave Equation](#Finite-Difference-Methods-for-the-One-Dimensional-Wave-Equation)
    * [Explicit Scheme](#Explicit-Scheme)
    * [Implicit Scheme](#Implicit-Scheme)
    * [Fourth-Order Scheme](#Fourth-Order-Scheme)

### Finite-Difference Methods for the One Dimensional Wave Equation

A one-dimensional for of the wave equation presented in (1) can be found as follows,
\begin{align}
\begin{cases}
\dfrac{\partial^2 u}{\partial t^2} = c^2(x) \dfrac{\partial^2 u}{\partial x^2}+f(x,t),&(x,t)\in (a,b)\times(T_1,T_2],\\
u(x,T_1)=g(x),&x\in [a,b],\\
\dfrac{\partial }{\partial t}u(x,T_1) = h(x),&x\in [a,b],\\
u(a,t) = f_a(t),&t \in[T_1,T_2],\\
u(b,t) = f_b(t),&t \in[T_1,T_2].
\end{cases}\qquad \qquad (1)
\end{align}

The following notations are introduced to generate an algorithm for the problem,

* $h=\Delta x=\frac{b-a}{N_x}$ and $\tau=\Delta t=\frac{T_2-T_1}{N_t}$ where $M$ and $N$ are positive integers,
* $u_{i}^{n}=u(ih,n\tau)$, $f_{i}^{n}=f(ih,n\tau)$ for $0\leq i \leq N_x$ and $0\leq n \leq N_t$,
* $c_{i}=c(ih)$ for $0\leq i \leq N_x$ and $\lambda=\frac{\tau}{h}$.

A general form of implicit Finite-Difference scheme of the problem (1) can be written as follows,
\begin{align}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}=
\frac{1}{h^2}c_{i}^2\left(\alpha \delta_x^2 u_{i}^{n+1} +\beta \delta_x^2 u_{i}^{n} +\gamma \delta_x^2 u_{i}^{n-1}\right)+f_{i}^{n},
\end{align}
where $\delta^2$ is the central difference operator and $\alpha,~\beta$ and $\gamma$ are constants such that $\alpha+\beta+\gamma=1$. It follows that
\begin{align}\label{eq1A1.001}
u_{i}^{n+1}-2u_{i}^{n}+u_{i}^{n-1} &= \alpha c_{i}^2 \lambda^2\left(  u_{i+1}^{n+1}-2u_{i}^{n+1}+u_{i-1}^{n+1}\right)
+\beta c_{i}^2 \lambda^2 \left( u_{i+1}^{n} -2 u_{i}^{n}+ u_{i-1}^{n}\right)
\notag \\ &
+\gamma c_{i}^2 \lambda^2  \left(u_{i+1}^{n-1}-2 u_{i}^{n-1}+u_{i-1}^{n-1}\right)+ \tau^2f_{i}^{n}.
\end{align}

This also can be simplified as follows,
\begin{align}
&-\alpha c_{i}^2 \lambda^2  \left(u_{i-1}^{n+1}+u_{i+1}^{n+1}\right)+\left(1 +2 \alpha c_{i}^2 \lambda^2 \right)u_{i}^{n+1}=\beta c_{i}^2 \lambda^2 \left(u_{i-1}^{n}+u_{i+1}^{n} \right)
\notag\\ &
+2\left(1 -\beta c_{i}^2 \lambda^2 \right)u_{i}^{n}+\gamma c_{i}^2 \lambda^2 \left(u_{i-1}^{n-1}+u_{i+1}^{n-1} \right) -\left(1 +2 \gamma c_{i}^2 \lambda^2 \right)u_{i}^{n-1}+\tau^2f_{i}^{n}.
\qquad
\qquad (2)
\end{align}

The following matrices are introduced to write down a matrix form of the equation (2),
$$
A=\begin{bmatrix}
1 +2 \alpha c_{1}^2 \lambda^2     &   -\alpha c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0
\\
-\alpha c_{2}^2 \lambda^2              &1 +2 \alpha c_{2}^2 \lambda^2    & -\alpha c_{2}^2 \lambda^2             & 0                   &  \vdots         &\vdots
\\
0                       &-\alpha c_{3}^2 \lambda^2              &1 +2 \alpha c_{3}^2 \lambda^2    & -\alpha c_{3}^2 \lambda^2               &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & -\alpha c_{M-3}^2 \lambda^2             & 1 +2 \alpha c_{M-3}^2 \lambda^2         &-\alpha c_{M-3}^2 \lambda^2  & 0
\\
\vdots                  &\vdots                      & 0                 & -\alpha c_{N_x-2}^2 \lambda^2             & 1 +2 \alpha c_{N_x-2}^2 \lambda^2         &-\alpha c_{N_x-2}^2 \lambda^2
\\
0                       &0                      & \dots                 & 0               & -\alpha c_{N_x-1}^2 \lambda^2     &  1 +2 \alpha c_{N_x-1}^2 \lambda^2
\end{bmatrix},
$$
$$
B=\begin{bmatrix}
2 -2 \beta c_{1}^2 \lambda^2     &   \beta c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0
\\
\beta c_{2}^2 \lambda^2              &2 -2 \beta c_{2}^2 \lambda^2    & \beta c_{2}^2 \lambda^2             & 0                   &  \vdots         &\vdots
\\
0                       &\beta c_{3}^2 \lambda^2              &2 -2 \beta c_{3}^2 \lambda^2    & \beta c_{3}^2 \lambda^2               &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & \beta c_{M-3}^2 \lambda^2             & 2 -2 \beta c_{M-3}^2 \lambda^2         &\beta c_{M-3}^2 \lambda^2  & 0
\\
\vdots                  &\vdots                      & 0                 & \beta c_{N_x-2}^2 \lambda^2             & 2 -2 \beta c_{N_x-2}^2 \lambda^2         &\beta c_{N_x-2}^2 \lambda^2
\\
0                       &0                      & \dots                 & 0               & \beta c_{N_x-1}^2 \lambda^2     &  2 -2 \beta c_{N_x-1}^2 \lambda^2
\end{bmatrix},
$$
$$
C=\begin{bmatrix}
-1-2 \gamma c_{1}^2 \lambda^2     &   \gamma c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0
\\
\gamma c_{2}^2 \lambda^2              &-1-2 \gamma c_{2}^2 \lambda^2    & \gamma c_{2}^2 \lambda^2             & 0                   &  \vdots         &\vdots
\\
0                       &\gamma c_{3}^2 \lambda^2              &-1-2 \gamma c_{3}^2 \lambda^2    & \gamma c_{3}^2 \lambda^2               &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & \gamma c_{M-3}^2 \lambda^2             & -1-2 \gamma c_{M-3}^2 \lambda^2         &\gamma c_{M-3}^2 \lambda^2  & 0
\\
\vdots                  &\vdots                      & 0                 & \gamma c_{N_x-2}^2 \lambda^2             & -1-2 \gamma c_{N_x-2}^2 \lambda^2         &\gamma c_{N_x-2}^2 \lambda^2
\\
0                       &0                      & \dots                 & 0               & \gamma c_{N_x-1}^2 \lambda^2     &  -1-2 \gamma c_{N_x-1}^2 \lambda^2
\end{bmatrix},
$$
and also
$$
\mathbf{u}^n=\begin{bmatrix}
u_{1}^{n} \\
u_{2}^{n}\\
\vdots\\
u_{N_x-2}^{n}\\
u_{N_x-1}^{n}
\end{bmatrix},~
\mathbf{b}^n=
\begin{bmatrix}
c_{1}^2  \lambda^2 \left(\alpha u_{0}^{n+1}+\beta u_{0}^{n}+\gamma u_{0}^{n-1}\right) \\
0\\
\vdots\\
0\\
c_{N_x-1}^2  \lambda^2 \left(\alpha u_{N_x}^{n+1}+\beta u_{N_x}^{n}+\gamma u_{N_x}^{n-1}\right)
\end{bmatrix},~
\mathbf{F}^n=\tau^2
\begin{bmatrix}
f_{1}^{n}\\
f_{2}^{n}\\
\vdots\\
f_{N_x-2}^{n}\\
f_{N_x-1}^{n}
\end{bmatrix}.
$$
Therefore, the following matrix form of equation (2) can be stated based as follows,
\begin{equation}
A \mathbf{u}^{n+1}=B \mathbf{u}^{n}+C \mathbf{u}^{n-1}+\mathbf{b}^n+\mathbf{F}^n,
\end{equation}
which implies
\begin{equation}
\mathbf{u}^{n+1}=A^{-1}\left(B \mathbf{u}^{n}+C \mathbf{u}^{n-1}+\mathbf{b}^n+ \mathbf{F}^n\right).
\end{equation}

Finite differnce method for the wave equation in time usually require computing the solution on the first two time steps. The solution on the second time step, $u(x,T_1+k)$, can be approximated as follows by means of Taylor series,
\begin{align}\label{eq1A1.06}
u(x,T_1+\tau)&=u(x,T_1)+u_t(x,T_1)\tau+\frac{1}{2} u_{tt}(x,T_1)\tau^2+\frac{1}{6} u_{ttt}(x,T_1)\tau^3
+\frac{1}{24} u_{tttt}(x,T_1)\tau^4
+\mathcal{O}(\tau^5)
\notag \\ &
=g(x)+s(x)\tau+\frac{1}{2} u_{tt}(x,T_1)\tau^2+\frac{1}{6} u_{ttt}(x,T_1)\tau^3+\frac{1}{24} u_{tttt}(x,T_1)\tau^4+\mathcal{O}(\tau^5)
\notag \\ &
=g(x)+s(x)\tau+\frac{1}{2}[c^2(x) \Delta g(x)+f(x,T_1)]\tau^2
+\frac{1}{6} [c^2(x) \Delta s(x)+f_t(x,T_1)]\tau^3
\notag\\ &
+\frac{1}{24}[c^2(x) \Delta [c^2(x) \Delta g(x)+f(x,T_1)] +f_{tt}(x,T_1)]\tau^4
+\mathcal{O}(\tau^5).
\end{align}

### Explicit Scheme
First, consider $\alpha=\gamma=0$ and $\beta=1$ in the equation (2) to get an **explicit** form of the algorithm. It follows that,
$$u_{i}^{n+1}=c_{i}^2 \lambda^2 \left(u_{i-1}^{n}+u_{i+1}^{n} \right)+2\left(1 -c_{i}^2 \lambda^2 \right)u_{i}^{n}-u_{i}^{n-1}+\tau^2 f_{i}^{n}.
$$
A matrix form of the (2) can be found as follows,
$$
\mathbf{u}^{n+1}=
\begin{bmatrix}
2 -2c_{1}^2 \lambda^2    &   c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0\\
c_{2}^2 \lambda^2              &2 -2c_{2}^2 \lambda^2    & c_{2}^2 \lambda^2             & 0                   &  \dots         &\vdots \\
0                       &c_{3}^2 \lambda^2             & \ddots                 & \ddots              &0         &0\\
0                       &0            & \ddots                 & \ddots              &0         &0\\
\vdots                        &\vdots           & \ddots                 & \ddots              &c_{M-3}^2 \lambda^2          &0\\
\vdots                  &\vdots                      & 0                & c_{N_x-2}^2 \lambda^2              & 2 -2c_{N_x-2}^2 \lambda^2        &c_{N_x-2}^2 \lambda^2 \\
0                       &0                      & \dots                 & 0               & c_{N_x-1}^2 \lambda^2     &  2 -2c_{N_x-1}^2 \lambda^2
\end{bmatrix}
\mathbf{u}^{n}
- \mathbf{u}^{n-1}
+\begin{bmatrix}
c_{1}^2 \lambda^2  u_{0}^{n} \\
0\\
\vdots\\
0\\
C_{N_x-1}^2 u_{N_x}^{n}
\end{bmatrix}
+\tau^2
\begin{bmatrix}
f_{1}^{n}\\
f_{2}^{n}\\
\vdots\\
f_{N_x-2}^{n}\\
f_{N_x-1}^{n}
\end{bmatrix},
$$
which also can be written in the following form,
\begin{equation}\label{eq1A11.03}
\mathbf{u}^{n+1}=B \mathbf{u}^{n}- \mathbf{u}^{n-1}+\mathbf{b}^n+ \mathbf{F}^n.
\end{equation}

### Implicit Scheme
Similarly, consider $\alpha=\gamma=\frac{1}{4}$ and $\beta=\frac{1}{2}$ in the equation (2) to get an **implicit** form of the algorithm. Therefore,
\begin{align}\label{eq1A11.04}
&-\frac{1}{4} c_{i}^2 \lambda^2  \left(u_{i-1}^{n+1}+u_{i+1}^{n+1}\right)+\left(1 +\frac{1}{2} c_{i}^2 \lambda^2 \right)u_{i}^{n+1}=\frac{1}{2} c_{i}^2 \lambda^2 \left(u_{i-1}^{n}+u_{i+1}^{n} \right)+\left(2 -c_{i}^2 \lambda^2 \right)u_{i}^{n}
\notag\\&
+\frac{1}{4} c_{i}^2 \lambda^2 \left(u_{i-1}^{n-1}+u_{i+1}^{n-1} \right) -\left(1 + \frac{1}{2} c_{i}^2 \lambda^2 \right)u_{i}^{n-1}+\tau^2 f_{i}^{n}.
\end{align}
This also can be expressed in the following matrix form,
\begin{equation}\label{eq1A11.05}
\mathbf{u}^{n+1}=A^{-1}\left(B \mathbf{u}^{n}+C \mathbf{u}^{n-1}+\mathbf{b}^n +\mathbf{F}_{n}\right),
\end{equation}
where
$$
A=\begin{bmatrix}
1 +\frac{1}{2} c_{1}^2 \lambda^2     &   -\frac{1}{4} c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0\\
-\frac{1}{4} c_{2}^2 \lambda^2              &1 +\frac{1}{2} c_{2}^2 \lambda^2    & -\frac{1}{4} c_{2}^2 \lambda^2             & 0                   &  \dots         &\vdots \\
0                       &-\frac{1}{4} c_{3}^2 \lambda^2             & \ddots                 & \ddots              &0         &0\\
0                       &0            & \ddots                 & \ddots              &0         &0\\
\vdots                        &\vdots           & \ddots                 & \ddots              &-\frac{1}{4} c_{M-3}^2 \lambda^2          &0\\
\vdots                  &\vdots                      & 0                 & -\frac{1}{4} c_{N_x-2}^2 \lambda^2             & 1 +\frac{1}{2} c_{N_x-2}^2 \lambda^2         &-\frac{1}{4} c_{N_x-2}^2 \lambda^2 \\
0                       &0                      & \dots                 & 0               & -\frac{1}{4} c_{N_x-1}^2 \lambda^2     &  1 +\frac{1}{2} c_{N_x-1}^2 \lambda^2
\end{bmatrix},
$$
$$
B=\begin{bmatrix}
2 - c_{1}^2 \lambda^2    &   \frac{1}{2} c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0\\
\frac{1}{2} c_{2}^2 \lambda^2              &2 - c_{2}^2 \lambda^2    & \frac{1}{2} c_{2}^2 \lambda^2             & 0                   &  \dots         &\vdots \\
0                       &\frac{1}{2} c_{3}^2 \lambda^2             & \ddots                 & \ddots              &0         &0\\
0                       &0            & \ddots                 & \ddots              &0         &0\\
\vdots                        &\vdots           & \ddots                 & \ddots              &\frac{1}{2} c_{M-3}^2 \lambda^2          &0\\
\vdots                  &\vdots                      & 0                & \frac{1}{2} c_{N_x-2}^2 \lambda^2              & 2 - c_{N_x-2}^2 \lambda^2        &\frac{1}{2} c_{N_x-2}^2 \lambda^2 \\
0                       &0                      & \dots                 & 0               & \frac{1}{2} c_{N_x-1}^2 \lambda^2     &  2 - c_{N_x-1}^2 \lambda^2
\end{bmatrix}=2(2I-A),
$$
$$
C=\begin{bmatrix}
-1 -\frac{1}{2} c_{1}^2 \lambda^2     &   \frac{1}{4} c_{1}^2 \lambda^2          & 0                      & \dots                   &  \dots         &0\\
\frac{1}{4} c_{2}^2 \lambda^2              &-1 -\frac{1}{2} c_{2}^2 \lambda^2    & \frac{1}{4} c_{2}^2 \lambda^2             & 0                   &  \dots         &\vdots \\
0                       &\frac{1}{4} c_{3}^2 \lambda^2             & \ddots                 & \ddots              &0         &0\\
0                       &0            & \ddots                 & \ddots              &0         &0\\
\vdots                        &\vdots           & \ddots                 & \ddots              &\frac{1}{4} c_{M-3}^2 \lambda^2          &0\\
\vdots                  &\vdots                      & \ddots                 & \frac{1}{4} c_{N_x-2}^2 \lambda^2              & -1 -\frac{1}{2} c_{N_x-2}^2 \lambda^2         &\frac{1}{4} C^2(x)\\
0                       &0                      & \dots                 & 0               & \frac{1}{4} c_{N_x-1}^2 \lambda^2     &  -1 -\frac{1}{2} c_{N_x-1}^2 \lambda^2
\end{bmatrix}=-A,
$$
and also
$$
\mathbf{b}^n=
\begin{bmatrix}
\frac{1}{4} c_{1}^2 \lambda^2  u_{0}^{n+1}+\frac{1}{2} c_{1}^2 \lambda^2  u_{0}^{n}+\frac{1}{4} c_{1}^2 \lambda^2  u_{0}^{n-1} \\
0\\
\vdots\\
0\\
\frac{1}{4} c_{N_x-1}^2 \lambda^2  u_{N_x}^{n+1}+\frac{1}{2} c_{N_x-1}^2 \lambda^2  u_{N_x}^{n}+\frac{1}{4} c_{N_x-1}^2 \lambda^2  u_{N_x}^{n-1}
\end{bmatrix},~
\mathbf{F}^n=\tau^2
\begin{bmatrix}
f_{1}^{n}\\
f_{2}^{n}\\
\vdots\\
f_{N_x-2}^{n}\\
f_{N_x-1}^{n}
\end{bmatrix}.
$$

**Theorem (Stability of the schemes)**

Assume that the solution of the one dimensional acoustic wave equation (1) is sufficiently smooth.
The stability criteria of the numerical scheme introduced in (2) can be derived as follows,
\begin{equation}
\label{eq.1D-Stbl}
\tau < \frac{1}{\sqrt{(1-4\alpha)}}\frac{h}{\max\limits_{0 < m \leq N_x} c_m}
\end{equation}
where $\alpha=\gamma$.
\end{theorem}
\begin{proof}
The Fourier transform of a function $u(x)$, which is defined on $\mathbb{R}$, is uniquely defined as follows,
\begin{equation}\label{eq1A3.01}
\hat{u}(\omega)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} e^{-i\omega x}u(x)dx.
\end{equation}
In addition, the Fourier inversion formula is given by the following equation,
\begin{equation}\label{eq1A3.02}
u(x)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} e^{i\omega x}\hat{u}(x)d\omega.
\end{equation}
Furthermore, the Fourier transform of a gird function such as $u_m$ for all integers $m$ is available as follows [[Strikwerda, 2004](https://my.siam.org/Store/Product/viewproduct/?ProductId=988)],
\begin{equation}\label{eq1A3.03}
\hat{u}(\xi)=\frac{1}{\sqrt{2\pi}}\sum_{m=-\infty}^{\infty} e^{-imh \xi}u_mh,\quad \xi \in [-\frac{\pi}{h},\frac{\pi}{h}],
\end{equation}
where $h=\Delta x$% and $\hat{v}(\frac{\pi}{h})=\hat{v}(-\frac{\pi}{h})$
. In addition, the Fourier inversion formula of $v$ for all integers $m$,
\begin{equation}\label{eq1A3.04}
u_m=\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{imh \xi}\hat{u}(\xi)d\xi.
\end{equation}
As a result, the Fourier inversion formula for $u_m^n$ for all $m$ and $n$ can be found as follows,
\begin{equation}\label{eq1A3.05}
u_m^n=\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{imh \xi}\hat{u}^n(\xi)d\xi,\qquad \qquad (3)
\end{equation}
where,
\begin{equation}\label{eq1A3.06}
\hat{u}^n(\xi)=\frac{1}{\sqrt{2\pi}}\sum_{m=-\infty}^{\infty} e^{-imh \xi}u_m^n,\quad \xi \in [-\frac{\pi}{h},\frac{\pi}{h}],
\end{equation}

First, let's perform the stability analyze of the explicit scheme using *Von Neumann stability analysis*. Consider the following explicit scheme,
\begin{equation}
\label{eq1A3.07}u_{m}^{n+1}=c_m^2 \lambda^2\left(u_{m-1}^{n}+u_{m+1}^{n} \right)+2\left(1 -c_m^2 \lambda^2\right)u_{m}^{n}-u_{m}^{n-1}+\tau^2 f_{Nm}^{n}.
\qquad \qquad (4)
\end{equation}
Without loss of generality, let $f_{m}^{n}=0$ for all values of $m$ and $n$. It follows from using the Fourier inversion formula (3) and (4),
\begin{align}
u_{m}^{n+1}&=c_m^2 \lambda^2\left(\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{i(m-1)h \xi}\hat{u}^n(\xi)d\xi+\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{i(m+1)h \xi}\hat{u}^n(\xi)d\xi \right)\notag\\
&+2\left(1 -c_m^2 \lambda^2\right)\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{imh \xi}\hat{u}^n(\xi)d\xi-\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{imh \xi}\hat{u}^{n-1}(\xi)d\xi
\notag\\ &
\label{eq1A3.08}
=\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{imh \xi}
\left[
c_m^2 \lambda^2(e^{-ih\xi}+e^{ih\xi})\hat{u}^n(\xi)+2\left(1 -c_m^2 \lambda^2\right)\hat{u}^n(\xi)-\hat{u}^{n-1}(\xi)
\right] d\xi,\qquad \qquad (5)
\end{align}
for all $1\leq m \leq N_x$ and $1\leq n \leq N_t$.

On the other hand, the Fourier inversion formula for $u_m^{n+1}$ can be found as follows,
\begin{equation}\label{eq1A3.09}
u_m^{n+1}=\frac{1}{\sqrt{2\pi}}\int_{-\frac{\pi}{h}}^{\frac{\pi}{h}} e^{imh \xi}\hat{u}^{n+1}(\xi)d\xi.\qquad \qquad (6)
\end{equation}
It follows from comparison of (5) and (6) and also the uniqueness of the Fourier transform,
\begin{equation}\label{eq1A3.10}
\hat{u}^{n+1}(\xi)=\left[c_m^2 \lambda^2(e^{-ih\xi}+e^{ih\xi})+2\left(1 -c_m^2 \lambda^2\right)\right] \hat{u}^n(\xi)-\hat{u}^{n-1}(\xi).
\end{equation}
Define \emph{the amplification factor} as follows,
\begin{equation}\label{eq1A3.11}
g=\frac{\hat{u}^{n+1}(\xi)}{\hat{u}^{n}(\xi)}.
\end{equation}
As a result,
\begin{equation}\label{eq1A3.12}
g+g^{-1}=2-4 c_m^2 \lambda^2 \sin^2 \frac{\theta}{2},
\end{equation}
where $\theta=h\xi$. It can be concluded that,
\begin{equation}
\label{eq1A3.13} g^{2}-2\varphi g+1=0.
\end{equation}
where $\varphi=1-2 c_m^2 \lambda^2 \sin^2 \frac{\theta}{2}$. The roots of the last quadratic equation are
\begin{equation}
\label{eq1A3.14} g_{\pm} =\varphi\pm \sqrt{\varphi^2-1}
\end{equation}

In case that $\varphi^2-1\geq0$, the quadratic equation will have two real roots and it follows form the definition of $\varphi$ that
$\varphi<-1$. This means that one of the roots is $g_{-}$,
\begin{equation}
\label{eq1A3.15} g_{-}=\varphi-\sqrt{\varphi^2-1}<\varphi<-1,
\end{equation}
which would lead to instability.

However, in case that $\varphi^2-1< 0$ or $|\varphi|< 1$ and the roots of the quadratic equation are complex,
\begin{equation}
\label{eq1A3.16} |g_{\pm}| =\left|\varphi\pm i\sqrt{1-\varphi^2}\right|=\sqrt{\varphi^2+1-\varphi^2}=1.
\end{equation}
It can be concluded that the stability is equivalent to $|\varphi|< 1$. Therefore,
\begin{equation}
\label{eq1A3.17} c_m^2 \lambda^2 \sin^2 \frac{\theta}{2} < 1,
\end{equation}
for any $\theta$. This means that $c_m \lambda < 1$ for $0\leq m \leq N_x$.

Following a stability analysis, the stability requirement here can be derived as follows,
\begin{equation}
\label{eq1A3.18} \tau \max\limits_{0 \leq m \leq N_x} c_m < h.
\end{equation}

Von Neumann stability analysis also can be applied to the general implicit scheme,
\begin{align}
&-\alpha  c_m^2 \lambda^2 \left(u_{N_x-1}^{n+1}+u_{m+1}^{n+1}\right)+\left(1 +2 \alpha  c_m^2 \lambda^2\right)u_{N_x}^{n+1}=\beta  c_m^2 \lambda^2\left(u_{N_x-1}^{n}+u_{m+1}^{n} \right)\notag\\
\label{eq1A3.19}&+2\left(1 -\beta  c_m^2 \lambda^2\right)u_{N_x}^{n}+\gamma  c_m^2 \lambda^2\left(u_{N_x-1}^{n-1}+u_{m+1}^{n-1} \right) -\left(1 +2 \gamma  c_m^2 \lambda^2\right)u_{N_x}^{n-1}+f_{N_x}^{n}\tau^2.
\end{align}

The analyze of stability is not sensitive to $f(x,y)$ due to the fact that $f(x,t)$ is known for $(x,t)\in[a,b]\times [T_1,T_2]$ and $f$ is assumed bounded. Therefore, without loss of generality, $f$ can be regarded zero for convenience. Similar to the previous case, it can be concluded that,
\begin{align}
&\left[-\alpha c_m^2 \lambda^2\left(e^{-ih\xi}+e^{ih\xi}\right)+\left(1 +2 \alpha  c_m^2 \lambda^2\right)\right]\hat{u}^{n+1}(\xi)
=\left[\beta c_m^2 \lambda^2\left(e^{-ih\xi}+e^{ih\xi}\right)\right.
\notag \\ &
\label{eq1A3.20}\left.+\left(2 -2 \beta  c_m^2 \lambda^2\right)\right]\hat{u}^{n}(\xi)
+\left[\gamma c_m^2 \lambda^2\left(e^{-ih\xi}+e^{ih\xi}\right)-\left(1+2 \gamma  c_m^2 \lambda^2\right)\right]\hat{u}^{n-1}(\xi).\qquad \qquad (6)
\end{align}


It follows from dividing the equation (6) by $\hat{u}^{n}(\xi)$ and the theorem assumption that $\alpha=\gamma$,

$$
\left[-\alpha c_m^2 \lambda^2\left(e^{-i\theta}+e^{i\theta}\right)+\left(1 +2 \alpha  c_m^2 \lambda^2\right)\right]g
=
\left[\beta c_m^2 \lambda^2\left(e^{-i\theta}+e^{i\theta}\right)+\left(2 -2 \beta  c_m^2 \lambda^2\right)\right]
+\left[\alpha c_m^2 \lambda^2\left(e^{-i\theta}+e^{i\theta}\right)-\left(1+2 \alpha  c_m^2 \lambda^2\right)\right]g^{-1},
\qquad \qquad (7)
$$
where $g$ is the amplification factor and $\theta=h\xi$. The equation (7) can be simplified as follows,
\begin{align}\label{eq1A3.22}
g^2-2\left[\frac{1-2\beta \varphi}{1+4\alpha \varphi}\right]g+1=0,
\end{align}
where $\varphi=c_m^2 \lambda^2\sin^2 \frac{\theta}{2}$. It follows that,
\begin{align}
\label{eq1A3.23}g^2+2\left(\frac{2(1-2\alpha) \varphi-1}{1+4\alpha \varphi}\right)g+1=0,
\qquad \qquad (8)
\end{align}
In case that $$\left(\frac{2(1-2\alpha) \varphi-1}{1+4\alpha \varphi}\right)^2-1\geq 0,$$ the quadratic equation (8) will have two distinct real roots and one of them is greater than 1. This would lead to instability.

Therefore, the following case is considered in which the roots of quadratic equation (8) are complex.
\begin{align}
\label{eq1A3.24}\left(\frac{2(1-2\alpha) \varphi-1}{1+4\alpha \varphi}\right)^2-1 < 0.
\end{align}
As a result,
\begin{align}
\label{eq1A3.25}-16 \alpha \varphi^2 +4 \varphi^2 - 4\varphi < 0.
\end{align}
Since $\varphi$ is non-zero,
\begin{align}
\label{eq1A3.26} (1-4\alpha)\varphi < 1.
\end{align}
This means,
\begin{align}
\label{eq1A3.27} (1-4\alpha)c_m^2 \lambda^2\sin^2 \frac{\theta}{2} < 1,
\end{align}
for any $\theta$. Therefore,
\begin{equation}
\label{eq1A3.28} \max\limits_{0 \leq m \leq N_x} c_m \lambda <  \frac{1}{\sqrt{(1-4\alpha)}} \text{ or }\tau < \frac{1}{\sqrt{(1-4\alpha)}}\frac{h}{\max\limits_{0 < m \leq N_x} c_m}.
\end{equation}

**Corollary**: 

The stability of the implicit method introduced in previous theorem can be derived by setting $\alpha=1/4$. It follows that,
\begin{equation}
\label{eq1A3.29} c_m \lambda <  \infty,
\end{equation}
which means that the implicit scheme is unconditionally stable.

**Theorem (Convergence)**
The numerical scheme (2) is the second order in both time and space considering $h=h_x=h_y$.

Without loss of generality, assume $f_{i}^{n}=0$ for all values of $i$ and $n$ in the equation (2). It follows that,
\begin{align}\label{1A2.01}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}= \frac{1}{h^2}c_{i}^2 \left(\alpha \delta_x^2 u_{i}^{n+1} +\beta \delta_x^2 u_{i}^{n} +\gamma \delta_x^2 u_{i}^{n-1}\right).
\qquad \qquad (9)
\end{align}
We know,
\begin{equation}\label{1A2.02}
\begin{cases}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}=\frac{d^2}{dt^2} u_{i}^{n}+\frac{1}{12}\frac{d^4}{dt^4} u_{i}^{n}\tau^2+\frac{1}{360}\frac{d^6}{dt^6} u_{i}^{n}\tau^4+O(\tau^6),\\
\frac{1}{h^2}\delta_x^2 u_{i}^{n}= \frac{d^2}{dx^2} u_{i}^{n}+\frac{1}{12}\frac{d^4}{dx^4} u_{i}^{n}h^2+\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4+O(h^6),
\end{cases}
\end{equation}
It follows from (9),
\begin{align}
&\frac{d^2}{dt^2} u_{i}^{n}+\frac{1}{12}\frac{d^4}{dt^4} u_{i}^{n}\tau^2+\frac{1}{360}\frac{d^6}{dt^6} u_{i}^{n}\tau^4+O(\tau^6) =
c_{i}^2\left[\alpha \frac{d^2}{dx^2} u_{i}^{n+1}+\beta\frac{d^2}{dx^2} u_{i}^{n}+\gamma \frac{d^2}{dx^2} u_{i}^{n-1} \right]
\notag \\ &
+\frac{c_{i}^2}{12}\left[\alpha\frac{d^4}{dx^4} u_{i}^{n+1}+\beta\frac{d^4}{dx^4} u_{i}^{n}+\frac{\gamma }{\Gamma}\frac{d^4}{dx^4} u_{i}^{n-1}  \right]h^2
\label{1A2.06}+ \frac{c_{i}^2}{360}\left[\alpha\frac{d^6}{dx^6} u_{i}^{n+1}+\beta\frac{d^6}{dx^6} u_{i}^{n}
+\frac{\gamma }{\Gamma}\frac{d^6}{dx^6} u_{i}^{n-1}   \right]h^4
+O(h^6).
\end{align}
Therefore,
\begin{align}\label{1A2.07}
&\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2} c_{i}^2 \delta_x^2 \left(\alpha  u_{i}^{n+1} +\beta  u_{i}^{n} +\gamma \ u_{i}^{n-1}\right)=
\frac{d^2}{dt^2} u_{i}^{n}- c_{i}^2\frac{d^2}{dx^2} \left[ \alpha u_{i}^{n+1}+\beta u_{i}^{n} + \gamma u_{i}^{n-1}\right]
\notag \\ &
+\frac{1}{12}\left[\frac{d^4}{dt^4} u_{i}^{n}\tau^2-c_{i}^2 \frac{d^4}{dx^4} \left(\alpha  u_{i}^{n+1}+\beta  u_{i}^{n}+\gamma u_{i}^{n-1}\right)h^2\right]
\notag \\ &
+\frac{1}{360}\left[\frac{d^6}{dt^6} u_{i}^{n}\tau^4-c_{i}^2 \frac{d^6}{dx^6} \left( \alpha  u_{i}^{n+1}+\beta u_{i}^{n}+\gamma  u_{i}^{n-1}\right)h^4\right]+O(h^6+\tau^6).
\qquad \qquad (10)
\end{align}

The order of the explicit scheme follows from setting $\alpha=\gamma=0$ and $\beta=1$ in (10),

\begin{equation}\label{1A2.08}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2}c_{i}^2\delta_x^2 u_{i}^{n}=
\frac{d^2}{dt^2} u_{i}^{n}- c_{i}^2\frac{d^2}{dx^2} u_{i}^{n}
+\frac{1}{12}\left[\frac{d^4}{dt^4} u_{i}^{n}\tau^2-c_{i}^2\frac{d^4}{dx^4} u_{i}^{n}h^2\right]+O(h^4+\tau^4).
\qquad \qquad (11)
\end{equation}

It follows from (11),
\begin{align}\label{1A2.10}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2}c_{i}^2\delta_x^2 u_{i}^{n}&=
\frac{d^2}{dt^2} u_{i}^{n}- c_{i}^2\frac{d^2}{dx^2} u_{i}^{n}
+\lim_{\tau\rightarrow 0} \left[\frac{1}{12}\left[\frac{d^4}{dt^4} u_{i}^{n}\tau^2-c_{i}^2\frac{d^4}{dx^4} u_{i}^{n}h^2\right]+O(h^4+\tau^4)\right],
\notag \\ &
=-\frac{c_{i}^2}{12}\frac{d^4}{dx^4} u_{i}^{n}h^2+O(h^4),
\end{align}
and setting $h=\mu \tau$ where $\mu$ is a known integer
\begin{align}\label{1A2.11}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2}c_{i}^2\delta_x^2 u_{i}^{n}&=
\frac{d^2}{dt^2} u_{i}^{n}- c_{i}^2\frac{d^2}{dx^2} u_{i}^{n}
+\frac{1}{12}\left[\frac{d^4}{dt^4} u_{i}^{n}-\mu^2c_{i}^2\frac{d^4}{dx^4} u_{i}^{n}\right]\tau^2+O(\tau^4),
\end{align}

Therefore, the explicit scheme is second order respect to both $h$ and $\tau$ and also the total order of this scheme is 2.

The order of the implicit scheme follows from setting $\alpha=\gamma=\frac{1}{4}$ and $\beta=\frac{1}{2}$ in (10),
\begin{align}
&\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2}c_{i}^2\left(\frac{1}{4} \delta_x^2 u_{i}^{n+1} -\frac{1}{2} \delta_x^2 u_{i}^{n} -\frac{1}{4} \delta_x^2 u_{i}^{n-1}\right)=
\notag \\ &
\label{1A2.12}\frac{1}{12}\left[\frac{d^4}{dt^4} u_{i}^{n}\tau^2-c_{i}^2 \left(\frac{1}{4} \frac{d^4}{dx^4} u_{i}^{n+1}+\frac{1}{2} \frac{d^4}{dx^4} u_{i}^{n}+\frac{1}{4} \frac{d^4}{dx^4} u_{i}^{n-1}\right)h^2\right]+O(h^4+\tau^4).
\qquad \qquad (12)
\end{align}
Similarly, after taking limits,

Similarly, it follows from applying limits on (12)
\begin{align}
\label{1A2.13}
&\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2}c_{i}^2\delta_x^2  \left(\frac{1}{4} u_{i}^{n+1} -\frac{1}{2}  u_{i}^{n} -\frac{1}{4} u_{i}^{n-1}\right)=
%\notag \\ &
-\frac{c_{i}^2}{12} \frac{d^4}{dx^4} \left(\frac{1}{4} u_{i}^{n+1}+\frac{1}{2} u_{i}^{n}+\frac{1}{4} u_{i}^{n-1}\right)h^2+O(h^4)
\\ &
\label{1A2.24}
\frac{1}{\tau^2}\delta_t^2 u_{i}^{n}- \frac{1}{h^2}c_{i}^2\delta_x^2  \left(\frac{1}{4} u_{i}^{n+1} -\frac{1}{2}  u_{i}^{n} -\frac{1}{4} u_{i}^{n-1}\right)=
\frac{1}{12}\frac{d^4}{dt^4} u_{i}^{n}\tau^2+O(\tau^4).
\end{align}

Hence, the implicit scheme is second order respect to both $h$ and $\tau$ and the total order of this scheme is also 2.
\end{proof}

### Fourth-Order Scheme

A higher-order scheme can be introduced as follows (the readers are encouraged to see [[Liao, 2008](https://www.sciencedirect.com/science/article/abs/pii/S0096300308007042)]),
\begin{align}
\label{eq1A5.01}\left(1+\frac{1^2}{12}\delta_t^2\right)^{-1}\frac{\delta_t^2}{\tau^2} u_{i}^{n}=
c^2_i \left(1+\frac{1}{12}\delta_x^2\right)^{-1}\frac{\delta_x^2}{h^2} u_{i}^{n}+f_{i}^{n},
\end{align}
where it will be shown that this scheme is 4th-order.

In this section, the scheme is analyzed and also an numerical algorithm is generated. This equation can be also written in the following form,
\begin{align}\label{eq1A5.02}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2_i}=
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}
+\left(1+\frac{1}{12}\delta_t^2\right)\left(1+\frac{1}{12}\delta_x^2\right)\frac{f_{i}^{n}}{c^2_i}.
\qquad \qquad (13)
\end{align}
The left hand side of the equation (13) can be expanded as follows,
\begin{align}\label{eq1A5.03}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2_i}&=
\frac{1}{12\tau^2}\left[ \left(\frac{u_{i+1}^{n+1}}{c^2_{i+1}}+10 \frac{u_{i}^{n+1}}{c^2_{i}} + \frac{u_{i-1}^{n+1}}{c^2_{i-1}}\right)
-2\left(\frac{u_{i+1}^{n}}{c^2_{i+1}}+10 \frac{u_{i}^{n}}{c^2_{i}} + \frac{u_{i-1}^{n}}{c^2_{i-1}}\right)\right.\notag\\
&+\left. \left(\frac{u_{i+1}^{n-1}}{c^2_{i+1}}+10 \frac{u_{i}^{n-1}}{c^2_{i}} + \frac{u_{i-1}^{n-1}}{c^2_{i-1}}\right)\right].
\end{align}

It follows that,
\begin{align}
\label{eq1A5.07}
&\left[\frac{u_{i+1}^{n+1}}{c^2_{i+1}}+10 \frac{u_{i}^{n+1}}{c^2_{i}} + \frac{u_{i-1}^{n+1}}{c^2_{i-1}}\right]
-2 \left[\frac{u_{i+1}^{n}}{c^2_{i+1}}+10 \frac{u_{i}^{n}}{c^2_{i}} + \frac{u_{i-1}^{n}}{c^2_{i-1}}\right]+\left[\frac{u_{i+1}^{n-1}}{c^2_{i+1}}+10 \frac{u_{i}^{n-1}}{c^2_{i}} + \frac{u_{i-1}^{n-1}}{c^2_{i-1}}\right]
\notag\\ &
=\lambda^2\left[u_{i+1}^{n+1}-2 u_{i}^{n+1} + u_{i-1}^{n+1}\right]+10\lambda^2\left[u_{i+1}^{n}-2u_{i}^{n} + u_{i-1}^{n}\right]
+\lambda^2 \left[u_{i+1}^{n-1}-2u_{i}^{n-1} + u_{i-1}^{n-1}\right]
\notag\\ &
+\tau^2\mathbf{F}_{i}^{n}.
\qquad \qquad (14)
\end{align}
where
\begin{align}
\label{eq1A5.06}\mathbf{F}_{i}^{n}&=
\frac{1}{144}\left[\frac{1}{c^2_{i+1}}\left(f_{i+1}^{n+1}+10 f_{i+1}^{n}+f_{i+1}^{n-1}\right)
+\frac{10}{c^2_{i}}(f_{i}^{n+1}+10f_{i}^{n}+f_{i}^{n-1})\right.\notag\\
&+\left.\frac{1}{c^2_{i-1}}(f_{i-1}^{n+1}+10f_{i-1}^{n}+f_{i-1}^{n-1})\right]
\end{align}
The equation (14) can be simplified as follows,

\begin{align}
\left(\frac{1}{c^2_{i+1}}-\lambda^2\right)u_{i+1}^{n+1}+\left(\frac{10}{c^2_{i}}+2\lambda^2\right)u_{i}^{n+1}+\left(\frac{1}{c^2_{i-1}}-\lambda^2\right)u_{i-1}^{n+1}
&=
\left(\frac{2}{c^2_{i+1}}+10\lambda^2\right)u_{i+1}^{n}+\left(\frac{20}{c^2_{i}}-20\lambda^2\right)u_{i}^{n}
+\left(\frac{2}{c^2_{i-1}}+10\lambda^2\right)u_{i-1}^{n}
\notag \\ &
-\left(\frac{1}{c^2_{i+1}}-\lambda^2\right)u_{i+1}^{n-1}
-\left(\frac{10}{c^2_{i}}+2\lambda^2\right)u_{i}^{n-1}-\left(\frac{1}{c^2_{i-1}}-\lambda^2\right)u_{i-1}^{n-1}
+\tau^2\mathbf{F}_{i}^{n}.
\end{align}

Consider the following matrices,
$$
A=\begin{bmatrix}
\frac{10}{c^2_{1}}+2\lambda^2 & \frac{1}{c^2_{2}}-\lambda^2 & 0 & 0 &\dots & \dots &0 \\
\frac{1}{c^2_{1}}-\lambda^2 & \frac{10}{c^2_{2}}+2\lambda^2 & \frac{1}{c^2_{3}}-\lambda^2 & 0 &  0 &\dots &\vdots \\
0 & \frac{1}{c^2_{2}}-\lambda^2 & \frac{10}{c^2_{3}}+2\lambda^2 & \frac{1}{c^2_{4}}-\lambda^2 & 0 & \dots &\vdots \\
0 & 0& \ddots & \ddots & \ddots & 0 & \vdots \\
\vdots & \dots &0 & \frac{1}{c^2_{M-4}}-\lambda^2 & \frac{10}{c^2_{M-3}}+2\lambda^2 & \frac{1}{c^2_{N_x-2}}-\lambda^2 & 0\\
\vdots & \dots & 0 &0 & \frac{1}{c^2_{M-3}}-\lambda^2 & \frac{10}{c^2_{N_x-2}}+2\lambda^2 & \frac{1}{c^2_{N_x-1}}-\lambda^2 \\
0 & \dots & \dots & 0 &0 & \frac{1}{c^2_{N_x-2}}-\lambda^2 & \frac{10}{c^2_{N_x-1}}+2\lambda^2\\
\end{bmatrix},
$$
$$
B=\begin{bmatrix}
\frac{20}{c^2_{1}}-20\lambda^2 & \frac{2}{c^2_{2}}+10\lambda^2 & 0 & 0 &\dots & \dots &0 \\
\frac{2}{c^2_{1}}+10\lambda^2 & \frac{20}{c^2_{2}}-20\lambda^2 & \frac{2}{c^2_{3}}+10\lambda^2 & 0 &  0 &\dots &\vdots \\
0 & \frac{2}{c^2_{2}}+10\lambda^2 & \frac{20}{c^2_{3}}-20\lambda^2 & \frac{2}{c^2_{4}}+10\lambda^2 & 0 & 0 &\vdots \\
0 & 0& \ddots & \ddots & \ddots & 0 & \vdots \\
\vdots & \dots &0 & \frac{2}{c^2_{M-4}}+10\lambda^2 & \frac{20}{c^2_{M-3}}-20\lambda^2 & \frac{2}{c^2_{N_x-2}}+10\lambda^2 & 0\\
\vdots & \dots & 0 &0 & \frac{2}{c^2_{M-3}}+10\lambda^2 & \frac{20}{c^2_{N_x-2}}-20\lambda^2 & \frac{2}{c^2_{N_x-1}}+10\lambda^2 \\
0 & \dots & \dots & 0 &0 & \frac{2}{c^2_{N_x-2}}+10\lambda^2 & \frac{20}{c^2_{N_x-1}}-20\lambda^2\\
\end{bmatrix},
$$
$$
C=-A
$$
and also
$$
\textbf{u}^n=\begin{bmatrix}
u_{1}^{n} \\
u_{2}^{n}\\
\vdots\\
u_{N_x-2}^{n}\\
u_{N_x-1}^{n}
\end{bmatrix},~
\textbf{b}^n=
\begin{bmatrix}
-\left(\frac{1}{c^2_{0}}-\lambda^2\right)f_{a}^{n+1}+\left(\frac{2}{c^2_{0}}+10\lambda^2\right)f_{a}^{n}
-\left(\frac{1}{c^2_{0}}-\lambda^2\right)f_{a}^{n-1}\\
0\\
\vdots\\
0\\
-\left(\frac{1}{c^2_{N_x}}-\lambda^2\right)f_{b}^{n+1}+\left(\frac{2}{c^2_{N_x}}+10\lambda^2\right)f_{b}^{n}
-\left(\frac{1}{c^2_{0}}-\lambda^2\right)f_{b}^{n-1} \\
\end{bmatrix},
 \textbf{F}^n=\tau^2
\begin{bmatrix}
\mathbf{F}_{1}^{n}\\
\mathbf{F}_{2}^{n}\\
\vdots\\
\mathbf{F}_{N_x-2}^{n}\\
\mathbf{F}_{N_x-1}^{n}
\end{bmatrix}.
$$
Therefore, the presented scheme can be expressed in the following matrix form,
\begin{align}
\label{eq1A5.11}\textbf{u}^{n+1}&=A^{-1}\left(B \textbf{u}^{n}+C \textbf{u}^{n-1}+\textbf{b}^n+ \textbf{F}^n\right).
\end{align}

**Theorem (Stability fo the fourth-order scheme)**
Assume that the solution of the two dimensional acoustic wave equation is sufficiently smooth.
The stability criteria of the numerical scheme can be derived as follows,
\begin{equation}
\label{eq.1D4th-Stbl}
\lambda \max_{x\in \Omega} \{|c(x)|\}=c\lambda < 1.
\end{equation}

First off, without loss of generality, assume that $f_{i}^n=0$ for all values of $i$ and $n$. Moreover, let $c=\max_{(x)\in \Omega} \{|c(x)|\}$. It follows that,
\begin{align}\label{eq1A7.01}
\left(1+ \frac{1-c^2\lambda^2}{12}\delta_{x}^{2} \right) \delta_{t}^{2} u_{m,j}^{n}&= c^2\lambda^2 \delta_{x}^{2} u_{m,j}^{n}.
\qquad \qquad (15)
\end{align}
Considering $\rho=\frac{1-c^2\lambda^2}{12}$, it follows from (15)
\begin{align}\label{eq1A7.02}
&\left(1+\rho \delta_x^2\right)
\left[ u_{i,j}^{n+1}+u_{i,j}^{n-1}\right]
-\left[
2\left(1+\rho \delta_x^2\right)
+c^2\lambda^2 \delta_x^2 \right]u_{m,j}^{n}=0.
\qquad \qquad (16)
\end{align}
Applying discrete Fourier transformation on (16) leads to the following result,
\begin{align}\label{eq1A7.03}
&\left(1+\rho \delta_x^2\right)e^{ih\left(m \xi\right)}
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]-
\left[2\left(1+\rho \delta_x^2\right)+c^2\lambda^2 \delta_x^2\right] e^{ih\left(m \xi\right)} \hat{u}^{n}=0,
\end{align}
which also can be expressed as follows, considering $\theta=h \xi$,
\begin{align}\label{eq1A7.04}
&\left(1+\rho \delta_x^2\right)e^{im \theta}\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
-\left[2\left(1+\rho \delta_x^2\right)e^{im \theta}
+c^2\lambda^2\delta_x^2e^{im \theta}\right]\hat{u}^{n}=0.
\end{align}
It follows that,
\begin{align*}
&e^{im \theta}
\left[ \rho \left(e^{\theta}+e^{-\theta}\right) +\left(1-2 \rho\right)\right]
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
\notag \\ &
-\left[
2e^{im \theta}
\left[ \rho \left(e^{\theta}+e^{-\theta}\right) +\left(1-2 \rho\right)\right]
+c^2\lambda^2 e^{im \theta}
\left[2\left(e^{\theta}+e^{-\theta}\right) -2\right]\right]
\hat{u}^{n}=0.
\end{align*}
This can be simplified as follows,
\begin{align}
&e^{im \theta}
\left( 2\rho \cos\theta +\left(1-2 \rho\right)\right)
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
\notag \\ &
-\left[
2e^{im \theta}
\left( 2\rho \cos\theta +\left(1-2 \rho\right)\right)
+c^2\lambda^2 e^{im \theta}
\left(2\cos\theta -2\right)
\right]
\hat{u}^{n}=0.
\end{align}
Dividing by $e^{im \theta}$,
\begin{align}\label{eq1A7.05}
&\left( 1-4\rho \sin^2\frac{\theta}{2}\right)
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
-\left[
2\left( 1-4\rho \sin^2\frac{\theta}{2}\right)
%\right. \notag \\ & \left.
-4c^2\lambda^2
\sin^2\frac{\theta}{2}\right]
\hat{u}^{n}=0,
\end{align}
It follow from the fact that $\rho=\frac{1-c^2\lambda^2}{12}$,
\begin{align*}
1-4\rho \sin^2\frac{\theta}{2}=1-\frac{1}{3} \sin^2\frac{\theta}{2}+\frac{c^2\lambda^2}{3} \sin^2\frac{\theta}{2}>0
\qquad \qquad (17)
\end{align*}

Furthermore, set $S=\sin\frac{\theta}{2}$. It follows from (17) that,
\begin{align}
&\left( 1-4\rho S^2\right)
\left[ \hat{u}^{n+1}+\hat{u}^{n-1}\right]
%\notag \\ &
-\left[
2\left( 1-4\rho S^2\right)
-4c^2\lambda^2S^2
\right]
\hat{u}^{n}=0.
\end{align}
Hence,
\begin{align}
&\hat{u}^{n+1}
+\left(4c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}
-2\right)
\hat{u}^{n}
+\hat{u}^{n-1}=0.
\end{align}
As a result,
\begin{align}\label{eq1A7.06}
\hat{u}^{n+1}-2\varphi(\theta)\hat{u}^{n}+\hat{u}^{n-1}=0,
\end{align}
where,
$\varphi(\theta)=1-2c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}$.
It follows that,
\begin{align}\label{eq1A7.07}
g^2-2\varphi(\theta)g+1=0,
\qquad \qquad (18)
\end{align}
where $g$ is the amplification factor.

In case that, $|\varphi(\theta)|\geq 1$
then quadratic equation (18) has two real roots and one of which is bigger than 1. This would lead to instability. Therefore,
The necessary and sufficient condition for the scheme to be stable can be considered as follows,
$$|\varphi(\theta)| < 1$$

On one hand, clearly, $S^2=\sin^2\frac{\theta}{2}\leq 1$. Hence, $\varphi(\theta,\theta_y)<  1$.

On the other hand,
\begin{align}\label{eq1A7.08}
1-2c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}> -1.
\end{align}
\begin{align}\label{eq1A7.09}
c^2\lambda^2\frac{S^2}
{\left( 1-4\rho S^2\right)}< 1.
\qquad \qquad (19)
\end{align}
Define,
\begin{align}\label{eq1A7.10}
\chi (S^2)=\frac{S^2}{\left( 1-4\rho S^2\right)}
\end{align}
If $c^2\lambda^2 = 1$ , $c^2\lambda^2 \chi( 1 ) = 1$, the inequality in the equation (19) is not satisfied, the method is not stable. Therefore, $c^2\lambda^2<1$ should be considered. It is easy to see that the numerator of $\chi(S^2)$ is an increasing function, whereas the
denominator of is a decreasing. It follows that, $\chi (S^2)$ takes its maximum at $(1)$. Hence,
\begin{align}\label{eq1A7.11}
c^2\lambda^2 \chi (1)<1.
\end{align}
This means,
\begin{align}\label{eq1A7.12}
\frac{c^2\lambda^2}{\left(\frac{2}{3}+\frac{1}{3}c^2\lambda^2 \right)}< 1.
\end{align}

Therefore, the scheme is stable if
\begin{align}\label{eq1A7.13}
\lambda \max_{x\in \Omega} \{|c(x)|\}=c\lambda < 1.
\end{align}
%\subsection{Dispersion Analysis of the Higher Order Schemes}\label{1A8}
%\% Should be written here.

**Theorem (Convergence of the scheme)**
The numerical scheme is fourth-order in both time and space considering $h=h_x=h_y$.
Without loss of generality, let's assume $f(x,t)=0$ for all $x$ and $t$. Moreover, let $c=\max_{(x)\in \Omega} \{|c(x)|\}$. As a result,
\begin{align}\label{eq1A6.01}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}=
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}.
\end{align}
We know,
\begin{align}\label{eq1A6.02}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}
%&=\left(1+\frac{h^2}{12}\delta_x^2\right)\left(\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}+\frac{1}{12}\frac{d^4}{dt^4} \frac{u_{i}^{n}}{c^2}\tau^2+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4+\mathcal{O}(\tau^6)\right)
%\notag\\ &
&=\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}+\frac{1}{12}\frac{d^4}{dt^4} \frac{u_{i}^{n}}{c^2}\tau^2+\frac{1}{12}\frac{d^4}{dx^2dt^2} \frac{u_{i}^{n}}{c^2}h^2
+\frac{1}{144}\frac{d^6}{dx^2dt^4} \frac{u_{i}^{n}}{c^2}h^2\tau^2\notag\\
&+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2}h^4+\mathcal{O}(h^6+\tau^6).
\end{align}
and
\begin{align}\label{eq1A6.03}
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}
%&=\left(1+\frac{h^2}{12}\delta_x^2\right)\left(\frac{d^2}{dx^2} u_{i}^{n+1}+\frac{1}{12}\frac{d^4}{dx^4} u_{i}^{n+1}h^2+\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n+1}h^4+\mathcal{O}(h^6)\right)
%\notag\\ &
&=\frac{d^2}{dx^2} u_{i}^{n}+\frac{1}{12}\frac{d^4}{dx^4} u_{i}^{n}h^2+\frac{1}{12}\frac{d^4}{dt^2dx^2} u_{i}^{n}\tau^2
+\frac{1}{144}\frac{d^6}{dt^2dx^4} u_{i}^{n}h^2\tau^2\notag\\
&+\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4
+\frac{1}{144}\frac{d^6}{dt^4dx^2} u_{i}^{n}\tau^4+\mathcal{O}(h^6+\tau^6)
\end{align}
Therefore,
\begin{align}\label{eq1A6.04}
&\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}-
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}=
\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}-\frac{d^2}{dx^2} u_{i}^{n}
+\frac{1}{144}\left(\frac{d^4}{dx^2dt^4} \frac{u_{i}^{n}}{c^2}-\frac{d^4}{dt^2dx^4} u_{i}^{n}\right)h^2\tau^2
\notag\\ &
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2}h^4
-\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4
+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4
-\frac{1}{144}\frac{d^6}{dt^4dx^2} u_{i}^{n}\tau^4
%\notag\\ &
+\mathcal{O}(h^6+\tau^6)
\end{align}

The order of method respect to $h$ can be easily obtained if $\tau \rightarrow 0$. In other words,

\begin{align}\label{eq1A6.05}
\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}-
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}&=
\frac{d^2}{dt^2} \frac{u_{i}^{n}}{c^2}-\frac{d^2}{dx^2} u_{i}^{n}
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2}h^4
\notag\\ &
-\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}h^4
+\mathcal{O}(h^6)
\end{align}

The following relationship between $h$ and $\tau$ is considered to find the order of method to $\tau$,
\begin{equation}\label{eq1A6.06}
h=\mu \tau
\end{equation}
where $\mu$ is a known integer.

Therefore,
\begin{align}\label{eq1A6.07}
&\left(1+\frac{1}{12}\delta_x^2\right)\frac{\delta_t^2}{\tau^2} \frac{u_{i}^{n}}{c^2}-
\left(1+\frac{1}{12}\delta_t^2\right)\frac{\delta_x^2}{h^2} u_{i}^{n}=
\frac{1}{144}\left(
\frac{d^4}{dx^2dt^4} \frac{u_{i}^{n}}{c^2}
-\frac{d^4}{dt^2dx^4} u_{i}^{n}
\right) \mu^2 \tau^4
\notag\\ &
+\frac{1}{144}\frac{d^6}{dx^4dt^2} \frac{u_{i}^{n}}{c^2} \mu^4 \tau^4
-\frac{1}{360}\frac{d^6}{dx^6} u_{i}^{n}\mu^4 \tau^4
+\frac{1}{360}\frac{d^6}{dt^6} \frac{u_{i}^{n}}{c^2}\tau^4
-\frac{1}{144}\frac{d^6}{dt^4dx^2} u_{i}^{n}\tau^4
+\mathcal{O}(\tau^6),
\end{align}
which means that the scheme is 4th order respect to $\tau$.

**Example**: Let's consider the following problem,
\begin{align}
\frac{\partial^2 u}{\partial t^2} &= \mathrm{e}^{-x} \frac{\partial^2 u}{\partial x^2}+{\mathrm{e}}^{t+x}-{\mathrm{e}}^t ,&(x,t)\in (0,1)\times(0,1],\\
u(x,0)&=\mathrm{e}^{x},&x\in [0,1],\\
\frac{\partial }{\partial t}u(x,0) &=  \mathrm{e}^{x},&x\in [0,1],\\
u(0,t) &= \mathrm{e}^{t} &t \in[0,1],\\
u(1,t) &= \mathrm{e}^{t+1},&t \in[0,1].
\end{align}
The exact solution corresponding to the problem can be found as follows,
\begin{equation}
u(x,t)=\mathrm{e}^{x+t}
\end{equation}

In [2]:
def Example01_ExSolver1D(Nx,Nt):
    # N = the number of mesh points for space
    # Nt = the number of mesh points for time
    # [a,b] and [T1,T2]
    a = 0.0
    b = 1.0
    T1 = 0.0
    T2 = 1.0
    # Functions
    C2 =lambda x: np.exp(-x)
    g=lambda x: np.exp(x)
    h=lambda x: np.exp(x)
    fa=lambda t: np.exp(t)
    fb=lambda t: np.exp(t+1)
    f=lambda x,t: np.exp(t + x) - np.exp(t)
    Ue=lambda x,t: np.exp(x+t)
    # Additional functions
    gxx=lambda x: np.exp(x);
    gxxxx=lambda x: np.exp(x);
    hxx=lambda x: np.exp(x);
    ft=lambda x,t: np.exp(t + x) - np.exp(t);
    ftt=lambda x,t: np.exp(t + x) - np.exp(t);
    # dx, dt and lambda^2
    dx =(b-a)/Nx
    dt =(T2-T1)/Nt
    l2=(dt/dx)**2
    # discretizing [a,b] and [T1,T2]
    xx=np.linspace(a, b, Nx+1)
    tt=np.linspace(T1, T2, Nt+1)
    # dicretizing c^2(x)
    CC=C2(xx)
    # u initial time value
    u0 = g(xx);
    u=np.zeros(Nx+1, dtype=float)
    u1=np.zeros(Nx+1, dtype=float)
    # Computing the solution at the first time step
    GG= g(xx)+h(xx)*dt+0.5*(C2(xx)*gxx(xx)+f(xx,T1))*(dt**2)+\
    (1/6)*(C2(xx)*hxx(xx)+ft(xx,T1))*(dt**3)+(1/24)*(C2(xx)*gxxxx(xx)+ftt(xx,T1))*(dt**4)
    u1[1:Nx]=GG[1:Nx]
    u1[0]=fa(tt[1])
    u1[Nx]=fb(tt[1])
    mid=list(range(1,Nx))
    midp=[i+1 for i in mid]
    midm=[i-1 for i in mid]
    # for loop
    for n in range(1,Nt):
        u=np.zeros(Nx+1, dtype=float)
        u[mid]=l2*CC[mid]*(u1[midp]-2*u1[mid]+u1[midm])+2*u1[mid]-u0[mid]+(dt**2)*f(xx[mid],tt[n])
        u[0]=fa(tt[n+1])
        u[Nx]=fb(tt[n+1])
        u0=u1
        u1=u
    # end of the loop
    # Norm
    Norm=LA.norm(u-Ue(xx,tt[Nt]), np.inf)
    return Norm

**Explicit scheme**:
The results of analyzing the numerical solution of the problem using the explicit scheme is available at following tables 

In [3]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=200;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_ExSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Nt,Norm,Ratio,Log
0,10,200,0.000477,0.000000,0.000000
1,20,200,0.000119,4.018871,2.006790
2,40,200,0.000028,4.226541,2.079477
3,80,200,0.000005,5.187090,2.374925


In [4]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=20;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_ExSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Nt,Norm,Ratio,Log
0,10,20,0.000265,0.000000,0.000000
1,20,40,0.000067,3.965520,1.987510
2,40,80,0.000017,3.987502,1.995485
3,80,160,0.000004,3.991334,1.996871


What stands out from the tables is that the order of the method respect to both in space and time is 2.

In [5]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=8;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_ExSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Nt,Norm,Ratio,Log
0,10,8,8.407262e-04,0.000000e+00,0.000000
1,20,16,3.312714e-02,2.537877e-02,-5.300234
2,40,32,1.302596e+06,2.543163e-08,-25.228801
3,80,64,1.578489e+23,8.252172e-18,-56.749932


In addition, table highlights the  result of the experiment in which the stability criteria was violated.

**Implicit scheme**:
The results of analyzing the numerical solution of the problem using the implicit scheme with $\alpha=\gamma=\frac{1}{4}$ and $\beta=\frac{1}{2}$ is available at the following tables

In [6]:
def Example01_ImSolver1D(Nx,Nt):
    # Nx = the number of mesh points for space
    # Nt = the number of mesh points for time
    # alpha, beta and gamma
    alpha=1/4
    beta=1/2
    gamma=1/4
    # [a,b] and [T1,T2]
    a = 0.0
    b = 1.0
    T1 = 0.0
    T2 = 1.0
    # Functions
    C2 =lambda x: np.exp(-x)
    g=lambda x: np.exp(x)
    h=lambda x: np.exp(x)
    fa=lambda t: np.exp(t)
    fb=lambda t: np.exp(t+1)
    f=lambda x,t: np.exp(t + x) - np.exp(t)
    Ue=lambda x,t: np.exp(x+t)
    # Additional functions
    gxx=lambda x: np.exp(x);
    gxxxx=lambda x: np.exp(x);
    hxx=lambda x: np.exp(x);
    ft=lambda x,t: np.exp(t + x) - np.exp(t);
    ftt=lambda x,t: np.exp(t + x) - np.exp(t);
    # dx, dt and lambda^2
    dx =(b-a)/Nx
    dt =(T2-T1)/Nt
    l2=(dt/dx)**2
    # discretizing [a,b] and [T1,T2]
    xx=np.linspace(a, b, Nx+1)
    tt=np.linspace(T1, T2, Nt+1)
    # dicretizing c^2(x)
    CC=C2(xx)
    # u initial time value
    u0 = g(xx);
    u=np.zeros(Nx+1, dtype=float)
    u1=np.zeros(Nx+1, dtype=float)
    # Computing the solution at the first time step
    GG= g(xx)+h(xx)*dt+0.5*(C2(xx)*gxx(xx)+f(xx,T1))*(dt**2)+\
    (1/6)*(C2(xx)*hxx(xx)+ft(xx,T1))*(dt**3)+(1/24)*(C2(xx)*gxxxx(xx)+ftt(xx,T1))*(dt**4)
    u1[1:Nx]=GG[1:Nx]
    u1[0]=fa(tt[1])
    u1[Nx]=fb(tt[1])
    # indeces
    mid=list(range(1,Nx))
    sup=list(range(1,Nx-1))
    sub=list(range(2,Nx))
    # Matrix A
    A=diags([-alpha*CC[sub]*l2,1+2*alpha*CC[mid]*l2,-alpha*CC[sup]*l2], [-1,0,1])
    # Matrix B
    B=diags([beta*CC[sub]*l2,2-2*beta*CC[mid]*l2,beta*CC[sup]*l2], [-1,0,1])
    # Matrix C
    C=diags([gamma*CC[sub]*l2,-1-2*gamma*CC[mid]*l2,gamma*CC[sup]*l2], [-1,0,1])
    # Fa and Fb
    FA= fa(tt)
    FB= fb(tt)
    for n in range(1,Nt):
        u=np.zeros(Nx+1, dtype=float)
        bb=np.zeros(len(mid), dtype=float)
        bb[0]=CC[1]*l2*(alpha*FA[n+1]+beta*FA[n]+gamma*FA[n-1])
        bb[Nx-2]=CC[Nx-1]*l2*(alpha*FB[n+1]+beta*FB[n]+gamma*FB[n-1])
        u[mid]=spsolve(A,B*u1[mid]+C*u0[mid]+bb+(dt**2)*f(xx[mid],tt[n]))
        u[0]=fa(tt[n+1])
        u[Nx]=fb(tt[n+1])
        u0=u1
        u1=u
    # end of the loop
    # Norm
    Norm=LA.norm(u-Ue(xx,tt[Nt]), np.inf)
    return Norm

In [7]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=200;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_ImSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

NameError: name 'diags' is not defined

In [ ]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=200;
Nt0=10;
Nx=np.asarray([Nx0 for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_ImSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

It can be seen that the order of the method respect to both in space and time is 2.

Furthermore, the implicit method is unconditionally stable. This can be clearly seen from the tables.

**The fourth-order scheme:**
The results of analyzing the numerical solution of the problem, using the explicit scheme, is available at the following tables.

In [ ]:
def Example01_4thSolver1D(Nx,Nt):
    # Nx = the number of mesh points for space
    # Nt = the number of mesh points for time
    # [a,b] and [T1,T2]
    a = 0.0
    b = 1.0
    T1 = 0.0
    T2 = 1.0
    # Functions
    C2 =lambda x: np.exp(-x)
    g=lambda x: np.exp(x)
    h=lambda x: np.exp(x)
    fa=lambda t: np.exp(t)
    fb=lambda t: np.exp(t+1)
    f=lambda x,t: np.exp(t + x) - np.exp(t)
    Ue=lambda x,t: np.exp(x+t)
    # Additional functions
    gxx=lambda x: np.exp(x);
    gxxxx=lambda x: np.exp(x);
    hxx=lambda x: np.exp(x);
    ft=lambda x,t: np.exp(t + x) - np.exp(t);
    ftt=lambda x,t: np.exp(t + x) - np.exp(t);
    # dx, dt and lambda^2
    dx =(b-a)/Nx
    dt =(T2-T1)/Nt
    l2=(dt/dx)**2
    # discretizing [a,b] and [T1,T2]
    xx=np.linspace(a, b, Nx+1)
    tt=np.linspace(T1, T2, Nt+1)
    # dicretizing c^2(x)
    CC=C2(xx)
    # u initial time value
    u0 = g(xx);
    u=np.zeros(Nx+1, dtype=float)
    u1=np.zeros(Nx+1, dtype=float)
    # Computing the solution at the first time step
    GG= g(xx)+h(xx)*dt+0.5*(C2(xx)*gxx(xx)+f(xx,T1))*(dt**2)+\
    (1/6)*(C2(xx)*hxx(xx)+ft(xx,T1))*(dt**3)+(1/24)*(C2(xx)*gxxxx(xx)+ftt(xx,T1))*(dt**4)
    u1[1:Nx]=GG[1:Nx]
    u1[0]=fa(tt[1])
    u1[Nx]=fb(tt[1])
    # indeces
    mid=list(range(1,Nx))
    sub=list(range(1,Nx-1))
    sup=list(range(2,Nx))
    midp=[i+1 for i in mid]
    midm=[i-1 for i in mid]
    # Matrix A
    A=diags([(1/CC[sub])-l2,(10/CC[mid])+2*l2,(1/CC[sup])-l2], [-1,0,1])
    # Matrix B
    B=diags([(2/CC[sub])+10*l2,(20/CC[mid])-20*l2,(2/CC[sup])+10*l2], [-1,0,1])
    # Matrix C
    C=-A
    # Fa and Fb
    FA= fa(tt)
    FB= fb(tt)
    for n in range(1,Nt):
        FF=((dt**2)/12)*((1/CC[midp])*(f(xx[midp],tt[n+1])+10*f(xx[midp],tt[n])+f(xx[midp],tt[n-1]))+\
        (10/CC[mid])*(f(xx[mid],tt[n+1])+10*f(xx[mid],tt[n])+f(xx[mid],tt[n-1]))+\
        (1/CC[midm])*(f(xx[midm],tt[n+1])+10*f(xx[midm],tt[n])+f(xx[midm],tt[n-1])))
        u=np.zeros(Nx+1, dtype=float)
        bb=np.zeros(len(mid), dtype=float)
        bb[0]=-(1/CC[0]-l2)*FA[n+1]+(2/CC[0]+10*l2)*FA[n]-(1/CC[0]-l2)*FA[n-1]
        bb[Nx-2]=-(1/CC[Nx]-l2)*FB[n+1]+(2/CC[Nx]+10*l2)*FB[n]-(1/CC[Nx]-l2)*FB[n-1]
        u[mid]=spsolve(A,B*u1[mid]+C*u0[mid]+bb+FF)
        u[0]=fa(tt[n+1])
        u[Nx]=fb(tt[n+1])
        u0=u1
        u1=u
        del FF
    # end of the loop
    # Norm
    Norm=LA.norm(u-Ue(xx,tt[Nt]), np.inf)
    return Norm

In [ ]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=200;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_4thSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

In [ ]:
it=4
Norm=np.zeros(it, dtype=float)
Nx0=10;
Nt0=20;
Nx=np.asarray([Nx0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n] =Example01_4thSolver1D(Nx[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': Nx, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data